# Department Store Association Rules Analysis

Author: Kathleen Young

Date created: Mon Feb  5 17:10:35 2018

This notebook includes code for an association rules analysis on two years of point of sale data from Dillard's deparment store. This data is proprietary and cannot be accessed by the public. For a more complete discussion of the results, please see my report Dillards_AR_Report.pdf.

### Import necessary packages

In [3]:
import pandas as pd
import psycopg2
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Connect to Postgres database

The point of sale data was too large to be downloaded locally. The data was accessed remotely through a Postgres database. This is a public repository so the username and password has been removed.

In [4]:
conn = None
try:
    conn = psycopg2.connect(
            "host='gallery.iems.northwestern.edu' dbname='iems308'user = '*****' password = '*****'")
except:
    print("Can't connect! :/")

### Data Exploration

The identifying information was removed from the column headers and had to be reassigned based on a data dictionary. Data exploration also includes queries into the quantity and type of data in the database.

In [5]:
#Decoding the pos.trnsact table
#c1: sku
#c2: store
#c3: register
#c4: trannum (transaction code)
#c5: seq (sequence number)?
#c6: sale date
#c7: stype (return or purchase)
#c8: quantity (item quantity of the transaction)
#c9: amt (total amount of the transaction charge to the customer)?
#c10: ? orgprice (original price of the item stock)
#c11: ? orgprice (original price of the item stock)  
#c12: interid (internal id)?
#c13: mic
#c14: always zero?

#Number of enteries in trnsact table
num_trnsact = pd.read_sql(
        "SELECT COUNT(*) FROM pos.trnsact",
        con=conn)
print("Number of entries in trnsact table:\n", num_trnsact)

#Number of stores
num_stores = pd.read_sql(
        "SELECT COUNT(*) FROM pos.strinfo",
        con=conn)
print("Number of stores:\n", num_stores)

#Number of SKUs
num_sku = pd.read_sql("SELECT COUNT(*) FROM pos.skuinfo",
                      con=conn)
print("Number of SKUs:\n", num_sku)

#Number of departments
num_depts = pd.read_sql("SELECT COUNT(*) FROM pos.deptinfo",
                        con=conn)
print("Number of departments:\n", num_depts)

#Departments table
depts = pd.read_sql("SELECT * FROM pos.deptinfo",
                    con=conn)

#Number SKUs in department 800 (CLINIQUE)
num_skus_800 = pd.read_sql("SELECT COUNT(*) FROM pos.skuinfo WHERE pos.skuinfo.dept = 800",
                       con=conn)
print("Number of SKUs in department 800 (CLINIQUE):\n", num_skus_800)

#Number SKUs in department 1704 (RALPH LAUREN)
num_skus_1704 = pd.read_sql("SELECT COUNT(*) FROM pos.skuinfo WHERE pos.skuinfo.dept = 1704",
                       con=conn)
print("Number of SKUs in department 1704 (RALPH LAUREN):\n", num_skus_1704)

#Number SKUs in departmetn 9306 (SPERRY)
num_skus_9306 = pd.read_sql("SELECT COUNT(*) FROM pos.skuinfo WHERE pos.skuinfo.dept = 9306",
                       con=conn)
print("Number SKUs in department 9306 (SPERRY):\n", num_skus_9306)

Number of entries in trnsact table:
        count
0  120916896
Number of stores:
    count
0    453
Number of SKUs:
      count
0  1048576
Number of departments:
    count
0     60
Number of SKUs in department 800 (CLINIQUE):
    count
0   1416
Number of SKUs in department 1704 (RALPH LAUREN):
    count
0  10195
Number SKUs in department 9306 (SPERRY):
    count
0   6883


### Creating additional tables

It was necessary to create a few extra tables for easier analysis. The amount of data was massive--far too much to process on my personal machine. Thus, smaller tables were created to aid analysis. These tables were also used to save data created by INNER JOINs.

These queries have been commented out, as it is only necessary to create the tables once.

In [6]:
#This was generally done in postgreSQL

#Create a table with all transactions from 2005-05-07
#pd.read_sql("CREATE TABLE kay498_schema.date AS SELECT * FROM pos.trnsact WHERE pos.trnsact.c6 = '2005-05-07'",
#                      con=conn)

#Create a new table with SKUs as strings so it can be INNER JOINed with pos.trnsact
#pd.read_sql("CREATE TABLE kay498_schema.new_table AS SELECT pos.skuinfo.sku, pos.skuinfo.dept FROM pos.skuinfo",
#                        con=conn)

#Create a table that INNER JOINs the new_table and pos.trnsact on SKUs
#(pos.trnsact now includes dept info)
#This did not work because I did not have enough space in my schema
#pd.read_sql("CREATE TABLE kay498_schema.trn_dept AS SELECT * FROM pos.trnsact INNER JOIN kay498_schema.new_table ON (pos.trnsact.c1 = kay498_schema.new_table.sku)",
#            con=conn)

#Create a table with the moline and department data
#CREATE TABLE kay498_schema.trnsact_dept AS
#SELECT * FROM kay498_schema.moline
#INNER JOIN kay498_schema.new_table ON (kay498_schema.moline.c1 = kay498_schema.new_table.sku)

### Convert the Postgres tables to Python dataframes

Creating these Python dataframes aided in analysis by reducing the amount of data that had to be processed on my machine. Each focuses on Moline, IL and a single department.

In [7]:
#Moline dataframe
df_moline = pd.read_sql("SELECT * FROM kay498_schema.moline",
                        con=conn)
#Moline Clinique dataframe
df_moline_clinique = pd.read_sql("SELECT * FROM kay498_schema.trnsact_dept WHERE kay498_schema.trnsact_dept.dept = 800",
                                 con = conn)
df_m_c_count = pd.read_sql("SELECT COUNT(*) FROM kay498_schema.trnsact_dept WHERE kay498_schema.trnsact_dept.dept = 800",
                                 con = conn)
#Moline Ralph Lauren dataframe
df_moline_rlauren = pd.read_sql("SELECT * FROM kay498_schema.trnsact_dept WHERE kay498_schema.trnsact_dept.dept = 1704",
                                 con = conn)
df_m_rl = pd.read_sql("SELECT COUNT(*) FROM kay498_schema.trnsact_dept WHERE kay498_schema.trnsact_dept.dept = 1704",
                                 con = conn)
#Moline dept 1100 dataframe
df_moline_1100 = pd.read_sql("SELECT * FROM kay498_schema.trnsact_dept WHERE kay498_schema.trnsact_dept.dept = 1100",
                                 con = conn)

### Creating the association rules

Frequent item sets (sets of items that are often bought together) were selected based on a minimum support of 5%--that is, the set occurred in at least 5% of transactions.

The rules were based on the lift of each of the frequent item sets. The lift is the ratio of the observed support to that expected if X and Y were independent. A lift of one implies that the probability of occurrence of the antecedent and that of the consequent are independent of each other. No rule can be drawn.

If the lift is > 1, the antecedant and consequent are somewhat dependent on each other. This may indicate meaningful rules.

If the lift is < 1, the presence of one item has negative effect on presence of other item and vice versa. The items could be substitutes for each other, but no association rule should be drawn.

As such, the min_threshold for establishing an association rule is 1.

### Moline, department 1100

First, a single department (Clinique) in a single city (Moline) was analyzed.

Start by one-hot encoding the dataset. By one-hot encoding, each column represents an item and each market basket is a row. The basket is represented in binary, with a 1 indicating that the item exists in the market basket and a 0 indicating that it doesn't.

Below is a function that encodes the quantity column to be 1 or 0. It is used to one-hot encode the dataset.

In [8]:
#Force the quantity column to only be between 0 and 1 (for one-hot)
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [9]:
#For a single dataframe
#Pick the dataframe
df = df_moline_1100

#Force the quantity column to be numeric (for one-hot)
df[['c8']] = df[['c8']].apply(pd.to_numeric)

#Create an "index" by combining all of the primary key columns into a single
#column.
df["index"] = df["c2"] + df["c3"]+ df["c4"]+ df["c5"]

#One-hot encode the whole thing
basket = (df.groupby(['index', 'c1'])['c8'].sum().unstack().reset_index().
          fillna(0).set_index('index'))
basket_sets = basket.applymap(encode_units)

Create the assocation rules by finding the frequent item sets based on support of 5% and make rules based on a lift greater than 1.

In [10]:
#Find frequent_itemsets with support of at least 5%
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)

#Make some rules based on a lift > 1
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

This results in only two rules, and they both involve the same items.

In [42]:
rules

,antecedants,consequents,support,confidence,lift
0,(173088 ...,(183088 ...,0.416667,0.12,1.2
1,(183088 ...,(173088 ...,0.100000,0.50,1.2


### Moline, all departments

The following code loops through all of the departments in the Moline Dillard's department store and returns a set of association rules.

First, prepare and one-hot encode the data

In [11]:
#Looping through departments
#Setting up variables
depts_list = depts.dept
df_dict = dict()
basket_dict = dict()
f_itemsets_dict = dict()
rules_dict = dict()
basket_sets_dict = dict()

#Making dataframes for each department
for dept in depts_list:
    df_dict[dept] = pd.read_sql(f"SELECT * FROM kay498_schema.trnsact_dept WHERE kay498_schema.trnsact_dept.dept = '{dept}'",
       con = conn)

#Making sure everything is numeric
for dept in depts_list:
    df_dict[dept][['c8']] = df_dict[dept][['c8']].apply(pd.to_numeric)

#Create an index for each new dataframe
for dept in depts_list:
    df_dict[dept]["index"] = df_dict[dept]["c2"] + df_dict[dept]["c3"]+ df_dict[dept]["c4"]+ df_dict[dept]["c5"]

#One-hot encode everything   
for dept in depts_list:
    if len(df_dict[dept].c1) > 0:
        basket_dict[dept] = (df_dict[dept].groupby(['index', 'c1'])['c8'].sum().unstack().
                   reset_index().fillna(0).set_index('index'))

#Force the one-hot values to be either 0 or 1
for dept in depts_list:
    if len(df_dict[dept].c1) > 0:
        basket_sets_dict[dept] = basket_dict[dept].applymap(encode_units)

Finally, the association rules are created for the Moline store. First, frequent item sets are established for each department in the Moline store. The specific departments with 10 or more frequent item sets are isolated. Association rules are created for these departments.

In [25]:
#Find the frequent item sets for all departments in Moline
for dept in depts_list:
    if len(df_dict[dept].c1) > 0:
        f_itemsets_dict[dept] = apriori(basket_sets_dict[dept], min_support=0.05, use_colnames=True)
        
#A list of the specific departments with 10 or more frequent item sets
high_sup = [1100, 3100, 4400, 6400, 7200, 7205, 8000, 9000]

#Loop through these and make rules for them
for dept in high_sup:
    rules_dict[dept] = association_rules(f_itemsets_dict[dept], metric="lift", min_threshold=1)

### The Association Rules
Below are the association rules for the five departments that had greater than 10 frequent item sets that resulted in more than zero rules. Unfortunately, the product names were not provided so they could not be matched to SKU number. The rules are given in terms of their SKU numbers instead.

In [26]:
#GARY F
rules_dict[1100]

,antecedants,consequents,support,confidence,lift
0,(173088 ...,(183088 ...,0.416667,0.12,1.2
1,(183088 ...,(173088 ...,0.100000,0.50,1.2


In [24]:
#BE2
rules_dict[3100]

,antecedants,consequents,support,confidence,lift
0,(2177157 ...,(2107157 ...,0.196581,0.260870,1.695652
1,(2107157 ...,(2177157 ...,0.153846,0.333333,1.695652
2,(4037330 ...,(2107157 ...,0.273504,0.218750,1.421875
3,(2107157 ...,(4037330 ...,0.153846,0.388889,1.421875
4,(2177157 ...,(3597708 ...,0.196581,0.347826,3.130435
5,(3597708 ...,(2177157 ...,0.111111,0.615385,3.130435
6,(2177157 ...,(4037330 ...,0.196581,0.347826,1.271739
7,(4037330 ...,(2177157 ...,0.273504,0.250000,1.271739
8,(2177157 ...,(5289751 ...,0.196581,0.260870,4.360248
9,(5289751 ...,(2177157 ...,0.059829,0.857143,4.360248


In [27]:
#BORA
rules_dict[4400]

,antecedants,consequents,support,confidence,lift
0,(4737469 ...,(5649840 ...,0.545455,0.166667,1.145833
1,(5649840 ...,(4737469 ...,0.145455,0.625000,1.145833
2,(4737469 ...,(6571028 ...,0.545455,0.100000,1.100000
3,(6571028 ...,(4737469 ...,0.090909,0.600000,1.100000
4,(5649840 ...,(6571028 ...,0.145455,0.375000,4.125000
5,(6571028 ...,(5649840 ...,0.090909,0.600000,4.125000


In [29]:
#COFFRET
rules_dict[7200]

,antecedants,consequents,support,confidence,lift
0,(6347532 ...,(1508645 ...,0.186047,0.312500,2.443182
1,(1508645 ...,(6347532 ...,0.127907,0.454545,2.443182
2,(6347532 ...,(2419753 ...,0.186047,0.312500,3.839286
3,(2419753 ...,(6347532 ...,0.081395,0.714286,3.839286


In [32]:
#ANNASUI
rules_dict[9000]

,antecedants,consequents,support,confidence,lift
0,(3428013 ...,(3638860 ...,0.081633,0.75,7.350
1,(3638860 ...,(3428013 ...,0.102041,0.60,7.350
2,(3718013 ...,(3428013 ...,0.204082,0.30,3.675
3,(3428013 ...,(3718013 ...,0.081633,0.75,3.675
4,(3428013 ...,(3848088 ...,0.081633,0.75,7.350
5,(3848088 ...,(3428013 ...,0.102041,0.60,7.350
6,(3428013 ...,(5798907 ...,0.081633,0.75,7.350
7,(5798907 ...,(3428013 ...,0.102041,0.60,7.350
8,(3718013 ...,(3578013 ...,0.204082,0.30,3.675
9,(3578013 ...,(3718013 ...,0.081633,0.75,3.675
